In [1]:
# For output clarity 
import warnings
warnings.filterwarnings('ignore')

In [2]:
# For cleaning data
import numpy as np
import pandas as pd

# For creating training and testing sets
from sklearn.model_selection import train_test_split

# For confirming im/balance
from collections import Counter

# For creating models
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# For validating models
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [3]:
#Set up import of columns
#Columns were at first chosen through visual evaluation. 
#If a column contained all '0' values or all '1' values, it was not included
#If a column contained the same value for every row, it was not included 
columns = [
    "loan_amnt", "term", "int_rate", "installment", "grade",
    "home_ownership", "annual_inc", "verification_status", "issue_d","loan_status",
    "dti", "delinq_2yrs", "inq_last_6mths", "open_acc", "pub_rec",
    "revol_bal", "total_acc", "initial_list_status", "out_prncp", "out_prncp_inv", 
    "total_pymnt", "total_pymnt_inv", "total_rec_prncp", "total_rec_int","last_pymnt_amnt", 
    "next_pymnt_d", "collections_12_mths_ex_med", "application_type", "tot_coll_amt", "tot_cur_bal",
    "open_acc_6m", "open_act_il", "open_il_12m", "open_il_24m", "mths_since_rcnt_il",
    "total_bal_il", "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl", "inq_last_12m", 
    "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy", "bc_util", "chargeoff_within_12_mths", 
    "delinq_amnt", "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", 
    "mort_acc", "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl", 
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl", "num_op_rev_tl", 
    "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats", "num_tl_90g_dpd_24m", "num_tl_op_past_12m", 
    "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies", "tot_hi_cred_lim", "total_bal_ex_mort", 
    "total_bc_limit", "total_il_high_credit_limit"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = 'LoanStats_2019Q1.csv'
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Any coloumns where all values were null we dropped
df = df.dropna(axis='columns', how='all')

# Any rows that contained null values were dropped
df = df.dropna()

# The `Issued` loan status was removed from the loan_status column
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# The interest rate was converted to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# The target column values were converted to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df

,loan_amnt,term,int_rate,installment,grade,home_ownership,annual_inc,verification_status,issue_d,loan_status,...,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,36 months,0.1719,375.35,C,RENT,66000.0,Source Verified,Mar-2019,low_risk,...,8.0,0.0,3.0,85.7,100.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,36 months,0.2000,929.09,D,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,...,17.0,0.0,6.0,91.2,50.0,1.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,60 months,0.2000,529.88,D,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,...,8.0,0.0,4.0,66.7,50.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,36 months,0.1640,353.55,C,RENT,92000.0,Verified,Mar-2019,low_risk,...,10.0,0.0,1.0,100.0,50.0,1.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,60 months,0.1474,520.39,C,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,...,14.0,0.0,1.0,100.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71022,12000.0,60 months,0.2727,368.37,E,RENT,63000.0,Not Verified,Jan-2019,low_risk,...,8.0,0.0,5.0,96.2,0.0,0.0,87939.0,60350.0,13500.0,62939.0
71023,5000.0,36 months,0.1992,185.62,D,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,...,5.0,0.0,3.0,100.0,0.0,1.0,30592.0,18611.0,3600.0,18492.0
71024,40000.0,36 months,0.0646,1225.24,A,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,...,21.0,0.0,3.0,98.2,12.5,0.0,1033574.0,95958.0,100800.0,78634.0
71025,16000.0,60 months,0.1131,350.36,B,MORTGAGE,72000.0,Verified,Jan-2019,low_risk,...,12.0,2.0,0.0,94.3,0.0,1.0,251486.0,74835.0,23000.0,63090.0


In [7]:
#Encoding feature columns with string values 
df_binary_encoded = pd.get_dummies(df, columns = [
    "term",
    "grade",
    "home_ownership",
    "issue_d",
    "verification_status",
    "initial_list_status",
    "next_pymnt_d",
    "application_type"
])

#Encoding target column
#low_risk: 0
#high_risk: 1
x = {'low_risk': 0}   
df_binary_encoded = df_binary_encoded.replace(x)
y = {'high_risk': 1}
df_binary_encoded = df_binary_encoded.replace(y)

pd.set_option('display.max_columns', None)
df_binary_encoded

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_amnt,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 36 months,term_ 60 months,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,0,27.24,0.0,0.0,8.0,0.0,1609.0,14.0,10204.87,10204.87,355.29,355.29,295.13,60.16,375.35,0.0,178.0,38199.0,3.0,4.0,2.0,4.0,3.0,36590.0,59.0,1.0,2.0,1553.0,58.0,3700.0,1.0,0.0,4.0,6.0,4775.0,447.0,77.7,0.0,0.0,93.0,96.0,3.0,3.0,0.0,20.0,1.0,0.0,1.0,2.0,1.0,1.0,10.0,3.0,3.0,2.0,8.0,0.0,3.0,85.7,100.0,0.0,65687.0,38199.0,2000.0,61987.0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,0,20.23,0.0,0.0,17.0,1.0,18368.0,34.0,24293.13,24293.13,873.53,873.53,706.87,166.66,929.09,0.0,165.0,228119.0,3.0,3.0,2.0,3.0,4.0,42273.0,86.0,4.0,8.0,5197.0,61.0,50400.0,4.0,9.0,3.0,13.0,13419.0,29502.0,28.4,0.0,0.0,87.0,286.0,1.0,1.0,2.0,1.0,11.0,3.0,7.0,11.0,8.0,13.0,14.0,13.0,18.0,11.0,17.0,0.0,6.0,91.2,50.0,1.0,271427.0,60641.0,41200.0,49197.0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,0,24.26,0.0,0.0,8.0,0.0,13247.0,21.0,19636.79,19636.79,485.44,485.44,363.21,122.23,529.88,0.0,0.0,45684.0,2.0,3.0,2.0,3.0,4.0,32437.0,75.0,2.0,3.0,2810.0,75.0,17500.0,0.0,4.0,1.0,6.0,5711.0,2839.0,62.1,0.0,0.0,159.0,156.0,7.0,4.0,0.0,7.0,7.0,6.0,2.0,5.0,2.0,7.0,8.0,5.0,13.0,5.0,8.0,0.0,4.0,66.7,50.0,0.0,60644.0,45684.0,7500.0,43144.0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,0,31.44,0.0,1.0,10.0,1.0,17996.0,23.0,9719.34,9719.34,335.33,335.33,280.66,54.67,353.55,0.0,0.0,68784.0,0.0,4.0,1.0,3.0,9.0,50788.0,66.0,0.0,1.0,6360.0,69.0,23000.0,3.0,5.0,4.0,4.0,6878.0,4899.0,75.1,0.0,0.0,100.0,89.0,24.0,9.0,0.0,34.0,0.0,0.0,4.0,6.0,4.0,4.0,17.0,6.0,6.0,6.0,10.0,0.0,1.0,100.0,50.0,1.0,99506.0,68784.0,19700.0,76506.0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,0,18.76,0.0,1.0,14.0,0.0,9091.0,28.0,21614.73,21614.73,484.36,484.36,385.27,99.09,520.39,0.0,0.0,177745.0,0.0,1.0,0.0,2.0,14.0,16828.0,84.0,1.0,1.0,3600.0,41.0,43100.0,2.0,0.0,2.0,4.0,13673.0,19699.0,28.6,0.0,0.0,67.0,138.0,11.0,11.0,4.0,11.0,0.0,0.0,4.0,6.0,7.0,11.0,6.0,12.0,18.0,6.0,14.0,0.0,1.0,100.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71022,12000.0,0.2727,368.37,63000.0,0,29.07,0.0,0.0,8.0,0

In [8]:
#df_binary_encoded.columns

# Split the Data into Training and Testing

In [9]:
# Creating the features
X = df_binary_encoded.drop(columns="loan_status")

# Creating the target
y = df_binary_encoded["loan_status"]

In [10]:
#X.describe()

In [11]:
# Checking the balance of our target values
#low_risk: 0
#high_risk: 1
y.value_counts()

0    70669
1      358
Name: loan_status, dtype: int64

In [12]:
# Normal train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

#Confirming imbalnace in the training set
#low_risk: 0
#high_risk: 1
Counter(y_train)

Counter({0: 52996, 1: 274})

# Ensemble Learners

Train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier.

Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
# Resampling the training data with the BalancedRandomForestClassifier
#Sci-kit learn version 1.0.2 must be installed for this module
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

In [17]:
# Fitting the model
brf_model = brf_model.fit(X_train, y_train)


AttributeError: can't set attribute

In [ ]:
# Making predictions using the testing data
predictions = brf_model.predict(X_test)

In [ ]:
# Calculating the balanced accuracy score
acc_score= balanced_accuracy_score(y_test, predictions)

In [ ]:
# Calculating the confusion matrix and creating df
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])


In [ ]:
#Displaying results 
# Displaying confusion matrix df 
display(cm_df)
#Displaying accuracy score
print(f"Accuracy Score : {acc_score}")
# Displaying the imbalanced classification report
# low_risk: 0
# high_risk: 1
print("Classification Report -- low_risk=0, high_risk=1")
print(classification_report_imbalanced(y_test, predictions))

In [ ]:
# Listing the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

### Easy Ensemble AdaBoost Classifier

In [14]:
# Training the EasyEnsembleClassifier
#Scikit-learn version 1.0 must be installed for this module
#This module conflicts with version 1.0.2
from imblearn.ensemble import EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [15]:
# Fitting the model
eec_model.fit(X_train, y_train)


EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [16]:
# Making predictions using the testing data
predictions = eec_model.predict(X_test)

In [17]:
# Calculating the balanced accuracy score
acc_score= balanced_accuracy_score(y_test, predictions)

In [18]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [19]:
#Displaying results 
# Displaying confusion matrix df 
display(cm_df)
#Displaying accuracy score
print(f"Accuracy Score : {acc_score}")
# Displaying the imbalanced classification report
# low_risk: 0
# high_risk: 1
print("Classification Report -- low_risk=0, high_risk=1")
print(classification_report_imbalanced(y_test, predictions))

,Predicted 0,Predicted 1
Actual 0,16478,1195
Actual 1,5,79


Accuracy Score : 0.9364294605976833
Classification Report -- low_risk=0, high_risk=1
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.93      0.94      0.96      0.94      0.88     17673
          1       0.06      0.94      0.93      0.12      0.94      0.88        84

avg / total       1.00      0.93      0.94      0.96      0.94      0.88     17757

